In [8]:
import pandas as pd
import re

In [9]:
from ddf_utils.str import to_concept_id

In [10]:
source = '../source/childcod_estimates_2000_2017.xls'

In [11]:
readme = open('../source/readme.txt').read()

In [12]:
print(readme)

WHO-MCEE estimates for child causes of death, 2000-2017.

Last updated November 2018


Please see full documentation at: http://www.who.int/healthinfo/global_burden_disease/en/

This file contains estimates at three levels: global (level==global), WHO regions (level==region) and country (level==country).

Explanation of variable names:
neo = neonatal period
post = postneonatal period
ufive = under five (i.e., neo+post)
nnd = total neonatal deaths
pnd = total postneonatal deaths

Counts, rates and fractions:
neo2 = number of deaths from cause #2
rneo2 = death rate from cause #2 (per 1000 livebirths). Uses UN-IGME live birth time series.
fneo2 = fraction of deaths (within neo, post or ufive categories) due to cause #2

*********************************************************************************************************
*  Cause cateogories:                                               
*  CH2 HIV/AIDS                                                                         
*  CH3 Di

In [13]:
data = pd.read_excel(source, sheet_name='estimates')

In [14]:
data.head()

,iso3,whocode,level,whoreg6,whoname,year,nnd,pnd,livebirths,neo2,...,fufive9,fufive10,fufive11,fufive12,fufive13,fufive15,fufive16,fufive17,neometh,pnmeth
0,NaN,NaN,global,NaN,NaN,2000,3.995121e+06,5.988005e+06,130371718,2041.707170,...,0.175832,0.137013,0.113365,0.048276,0.080158,0.057230,0.034224,0.044907,NaN,NaN
1,NaN,NaN,global,NaN,NaN,2001,3.889595e+06,5.800964e+06,130872035,2048.813152,...,0.173748,0.138253,0.113726,0.049423,0.080116,0.058283,0.034498,0.044844,NaN,NaN
2,NaN,NaN,global,NaN,NaN,2002,3.784917e+06,5.587293e+06,131555312,2036.302584,...,0.172129,0.139881,0.114217,0.050836,0.080242,0.059467,0.034715,0.044898,NaN,NaN
3,NaN,NaN,global,NaN,NaN,2003,3.681372e+06,5.447301e+06,132366525,1985.650357,...,0.169302,0.140658,0.113568,0.051814,0.079609,0.059969,0.034787,0.044603,NaN,NaN
4,NaN,NaN,global,NaN,NaN,2004,3.580942e+06,5.135495e+06,133260434,1931.609307,...,0.170302,0.144341,0.115115,0.053888,0.080416,0.062247,0.035696,0.049254,NaN,NaN


In [15]:
indicator_mapping = dict(
    nnd = 'deaths',
    pnd = 'deaths',
    livebirths = 'live_births',
    f = 'fraction_of_deaths',
    r = 'death_rate'
)

In [16]:
age_mapping = dict(
    neo = 'neo',
    post = 'post',
    ufive = 'u5',
    nnd = 'neo',
    pnd = 'post'
)

In [17]:
mapping = dict(
    neo = "neonatal period",
    post = "postneonatal period",
    # ufive = "under five (i.e., neo+post)",
    ufive = "under five",
    nnd = "total neonatal deaths",
    pnd = "total postneonatal deaths",
    livebirths = 'live births'
)

In [18]:
cats = [("2", "HIV/AIDS"),
("3", "Diarrhoeal diseases"),
("5", "Tetanus"),
("6", "Measles"),
("7", "Meningitis/encephalitis"),
("8", "Malaria"),
("9", "Acute respiratory infections"),
("10",  "Prematurity"),
("11",  "Birth asphyxia and birth trauma"),
("12",  "Sepsis and other infectious conditions of the newborn"),
("13",  "Other Group 1"),
("15",  "Congenital anomalies"),
("16",  "Other noncommunicable diseases"),
("17",  "Injuries")]

In [19]:
for c in cats:
    mapping[c[0]] = c[1]

In [20]:
name_regx = re.compile(r'([rf]?)([a-z]+)([0-9]*)')

In [21]:
def create_dimensions(ser_):
    """accepts a series with time/geo index, add cause/age_group dimension to it and return a dataframe"""
    ser = ser_.copy()
    name = ser.name
    c1, c2, c3 = name_regx.match(name).groups()
    if c2 in indicator_mapping:
        indicator = indicator_mapping[c2]
        if c2 in age_mapping:
            age_group = age_mapping[c2]
        else:
            age_group = None
    elif not c1:
        indicator = 'deaths'
        age_group = age_mapping[c2]
    else:
        indicator = indicator_mapping[c1]
        age_group = age_mapping[c2]
        
    if c3:
        cause = c3
    else:
        cause = None
        
    # print(indicator, age_group, cause)
    
    index_names = list(ser.index.names)
    ser.name = indicator
    df = ser.reset_index()
    
    if age_group:
        df['age_group'] = age_group
        index_names.append('age_group')
    if cause:
        df['cause'] = cause
        index_names.append('cause')
    
    df = df.set_index(index_names)
    return df

In [22]:
# datapoints

In [23]:
groups = data.groupby('level')

In [24]:
gbl = groups.get_group('global').copy()

In [25]:
gbl['global'] = 'global'

In [26]:
gbl = gbl.set_index(['global', 'year'])

In [27]:
gbl.head()

iso3  whocode   level whoreg6 whoname           nnd           pnd  \
global year                                                                     
global 2000  NaN      NaN  global     NaN     NaN  3.995121e+06  5.988005e+06   
       2001  NaN      NaN  global     NaN     NaN  3.889595e+06  5.800964e+06   
       2002  NaN      NaN  global     NaN     NaN  3.784917e+06  5.587293e+06   
       2003  NaN      NaN  global     NaN     NaN  3.681372e+06  5.447301e+06   
       2004  NaN      NaN  global     NaN     NaN  3.580942e+06  5.135495e+06   

             livebirths         neo2          neo3  ...   fufive9  fufive10  \
global year                                         ...                       
global 2000   130371718  2041.707170  41274.023867  ...  0.175832  0.137013   
       2001   130872035  2048.813152  39231.973209  ...  0.173748  0.138253   
       2002   131555312  2036.302584  37367.807967  ...  0.172129  0.139881   
       2003   132366525  1985.650357  35601.528086  ...  0.169302  0.140658   
       2004   133260434  1931.609307  33794.632342  ...  0.170302  0.144341   

             fufive11  fufive12  fufive13  fufive15  fufive16  fufive17  \
global year                                                               
global 2000  0.113365  0.048276  0.080158  0.057230  0.034224  0.044907   
       2001  0.113726  0.049423  0.080116  0.058283  0.034498  0.044844   
       2002  0.114217  0.050836  0.080242  0.059467  0.034715  0.044898   
       2003  0.113568  0.051814  0.079609  0.059969  0.034787  0.044603   
       2004  0.115115  0.053888  0.080416  0.062247  0.035696  0.049254   

             neometh  pnmeth  
global year                   
global 2000      NaN     NaN  
       2001      NaN     NaN  
       2002      NaN     NaN  
       2003      NaN     NaN  
       2004      NaN     NaN  

[5 rows x 136 columns]

In [28]:
gbl.loc[:, 'nnd':].head()

nnd           pnd  livebirths         neo2  \
global year                                                        
global 2000  3.995121e+06  5.988005e+06   130371718  2041.707170   
       2001  3.889595e+06  5.800964e+06   130872035  2048.813152   
       2002  3.784917e+06  5.587293e+06   131555312  2036.302584   
       2003  3.681372e+06  5.447301e+06   132366525  1985.650357   
       2004  3.580942e+06  5.135495e+06   133260434  1931.609307   

                     neo3           neo5  neo6          neo7  neo8  \
global year                                                          
global 2000  41274.023867  170829.053194     0  53599.677141     0   
       2001  39231.973209  151891.076635     0  53457.974292     0   
       2002  37367.807967  134126.178635     0  52959.472225     0   
       2003  35601.528086  117525.163406     0  52669.281283     0   
       2004  33794.632342  102151.514064     0  52162.494664     0   

                      neo9  ...   fufive9  fufive10  fufive11  fufive12  \
global year                 ...                                           
global 2000  307495.384236  ...  0.175832  0.137013  0.113365  0.048276   
       2001  293698.330841  ...  0.173748  0.138253  0.113726  0.049423   
       2002  280903.077278  ...  0.172129  0.139881  0.114217  0.050836   
       2003  269290.675465  ...  0.169302  0.140658  0.113568  0.051814   
       2004  257533.486472  ...  0.170302  0.144341  0.115115  0.053888   

             fufive13  fufive15  fufive16  fufive17  neometh  pnmeth  
global year                                                           
global 2000  0.080158  0.057230  0.034224  0.044907      NaN     NaN  
       2001  0.080116  0.058283  0.034498  0.044844      NaN     NaN  
       2002  0.080242  0.059467  0.034715  0.044898      NaN     NaN  
       2003  0.079609  0.059969  0.034787  0.044603      NaN     NaN  
       2004  0.080416  0.062247  0.035696  0.049254      NaN     NaN  

[5 rows x 131 columns]

In [29]:
gbl_ = gbl.loc[:, 'nnd':]

In [30]:
res = {}

for c in gbl_.columns:
    ser = gbl_[c].dropna()
    if not ser.empty:
        df = create_dimensions(ser)
        idx = tuple(df.index.names)
        if idx in res:
            res[idx].append(df)
        else:
            res[idx] = [df]

In [31]:
res_ = dict([(k, pd.concat(v)) for k, v in res.items()])

/Users/semio/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [32]:
for k, v in res_.items():
    by = '--'.join(k)
    for c in v:
        # indicator = v.columns[0]
        v[[c]].dropna().to_csv(f'../../ddf--datapoints--{c}--by--{by}.csv')

In [33]:
!open ../

In [34]:
gbl_.shape

(18, 131)

In [35]:
reg = groups.get_group('region').copy()

In [36]:
reg.head()

,iso3,whocode,level,whoreg6,whoname,year,nnd,pnd,livebirths,neo2,...,fufive9,fufive10,fufive11,fufive12,fufive13,fufive15,fufive16,fufive17,neometh,pnmeth
18,NaN,NaN,region,5_Emr,NaN,2000,5.370400e+05,5.763546e+05,13916665,5.650000,...,0.189895,0.157630,0.133337,0.056904,0.077299,0.058425,0.039940,0.047589,NaN,NaN
19,NaN,NaN,region,6_Wpr,NaN,2000,4.413232e+05,3.896404e+05,23383814,14.740000,...,0.199419,0.155877,0.146705,0.017782,0.076187,0.116674,0.051419,0.125682,NaN,NaN
20,NaN,NaN,region,1_Afr,NaN,2000,1.093053e+06,3.004498e+06,27143379,1831.253600,...,0.160086,0.089732,0.091056,0.033332,0.072031,0.031890,0.022256,0.031499,NaN,NaN
21,NaN,NaN,region,2_Amr,NaN,2000,2.030170e+05,2.145550e+05,15906970,38.673174,...,0.134754,0.210640,0.086137,0.060174,0.101796,0.155709,0.077181,0.065245,NaN,NaN
22,NaN,NaN,region,3_Sear,NaN,2000,1.615088e+06,1.686801e+06,39753939,144.239999,...,0.192376,0.169682,0.131090,0.070883,0.089459,0.051236,0.033170,0.036639,NaN,NaN


In [37]:
reg['region'] = reg['whoreg6'].str.lower()

In [38]:
reg_ = reg.set_index(['region', 'year']).loc[:, "nnd":]

In [39]:
reg_.head()

,,nnd,pnd,livebirths,neo2,neo3,neo5,neo6,neo7,neo8,neo9,...,fufive9,fufive10,fufive11,fufive12,fufive13,fufive15,fufive16,fufive17,neometh,pnmeth
region,year,,,,,,,,,,,,,,,,,,,,,
5_emr,2000,5.370400e+05,5.763546e+05,13916665,5.650000,5758.514565,43884.312199,0,11606.915558,0,38879.467439,...,0.189895,0.157630,0.133337,0.056904,0.077299,0.058425,0.039940,0.047589,NaN,NaN
6_wpr,2000,4.413232e+05,3.896404e+05,23383814,14.740000,3312.790323,5539.652784,0,1011.193203,0,59036.694999,...,0.199419,0.155877,0.146705,0.017782,0.076187,0.116674,0.051419,0.125682,NaN,NaN
1_afr,2000,1.093053e+06,3.004498e+06,27143379,1831.253600,10106.888873,55391.472726,0,19233.020052,0,83525.270969,...,0.160086,0.089732,0.091056,0.033332,0.072031,0.031890,0.022256,0.031499,NaN,NaN
2_amr,2000,2.030170e+05,2.145550e+05,15906970,38.673174,261.321459,633.595494,0,556.322208,0,8706.090874,...,0.134754,0.210640,0.086137,0.060174,0.101796,0.155709,0.077181,0.065245,NaN,NaN
3_sear,2000,1.615088e+06,1.686801e+06,39753939,144.239999,21590.050738,64972.566508,0,20279.355096,0,112414.753869,...,0.192376,0.169682,0.131090,0.070883,0.089459,0.051236,0.033170,0.036639,NaN,NaN


In [40]:
res = {}

for c in reg_.columns:
    ser = reg_[c].dropna()
    if not ser.empty:
        df = create_dimensions(ser)
        idx = tuple(df.index.names)
        if idx in res:
            res[idx].append(df)
        else:
            res[idx] = [df]
            
res_ = dict([(k, pd.concat(v)) for k, v in res.items()])

/Users/semio/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [41]:
for k, v in res_.items():
    by = '--'.join(k)
    for c in v:
        # indicator = v.columns[0]
        v[[c]].dropna().to_csv(f'../../ddf--datapoints--{c}--by--{by}.csv')

In [42]:
country = groups.get_group('country').copy()

In [43]:
country.head()

,iso3,whocode,level,whoreg6,whoname,year,nnd,pnd,livebirths,neo2,...,fufive9,fufive10,fufive11,fufive12,fufive13,fufive15,fufive16,fufive17,neometh,pnmeth
126,AFG,3010.0,country,5_Emr,Afghanistan,2000,60411.007812,66168.375000,994521,0.00,...,0.180532,0.140849,0.128570,0.038122,0.060966,0.036353,0.035646,0.039304,vamcm,vamcm
127,AFG,3010.0,country,5_Emr,Afghanistan,2001,61031.000000,70357.312500,1022927,0.00,...,0.177381,0.134222,0.127894,0.040031,0.060409,0.036344,0.035743,0.039140,vamcm,vamcm
128,AFG,3010.0,country,5_Emr,Afghanistan,2002,61469.007812,63188.746094,1048815,0.10,...,0.188849,0.137824,0.139844,0.048251,0.065392,0.039751,0.038875,0.042326,vamcm,vamcm
129,AFG,3010.0,country,5_Emr,Afghanistan,2003,61620.000000,56220.601562,1072468,0.10,...,0.201221,0.141473,0.154524,0.058993,0.071263,0.043784,0.042757,0.046023,vamcm,vamcm
130,AFG,3010.0,country,5_Emr,Afghanistan,2004,61509.000000,55688.328125,1093639,0.11,...,0.202044,0.137587,0.160958,0.070410,0.074271,0.045542,0.044108,0.047366,vamcm,vamcm


In [44]:
country['country'] = country['iso3'].str.lower()

In [45]:
country_ = country.set_index(['country', 'year']).loc[:, 'nnd':]

In [46]:
country_.neometh.unique()

array(['vamcm', 'vrmcm', 'vr data', 'other'], dtype=object)

In [47]:
# check if index have duplicated (because there are different method to calculate stat for a given year/country)
country_.index.has_duplicates

False

In [48]:
# good to go

In [49]:
country_ = country.set_index(['country', 'year']).loc[:, 'nnd':'fufive17']

In [50]:
country_.loc[country_.index.get_level_values('country') == 'zwe', 'ufive17']

country  year
zwe      2000    1029.800659
         2001    1105.600952
         2002    1201.685669
         2003    1313.412354
         2004    1441.439087
         2005    1575.163818
         2006    1720.766724
         2007    1798.146240
         2008    2079.992188
         2009    2180.393311
         2010    2218.140625
         2011    2256.319580
         2012    2275.126221
         2013    2324.449219
         2014    2189.076172
         2015    2081.308838
         2016    2042.850220
         2017    1850.951538
Name: ufive17, dtype: float64

In [51]:
res = {}

for c in country_.columns:
    ser = country_[c].dropna()
    if not ser.empty:
        df = create_dimensions(ser)
        idx = tuple(df.index.names)
        if idx in res:
            res[idx].append(df)
        else:
            res[idx] = [df]
            
res_ = dict([(k, pd.concat(v)) for k, v in res.items()])

/Users/semio/.pyenv/versions/3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [52]:
res.keys()

dict_keys([('country', 'year', 'age_group'), ('country', 'year'), ('country', 'year', 'age_group', 'cause')])

In [53]:
res[('country', 'year', 'age_group', 'cause')][-1].tail()

fraction_of_deaths
country year age_group cause                    
zwe     2013 u5        17               0.067116
        2014 u5        17               0.068167
        2015 u5        17               0.068754
        2016 u5        17               0.072404
        2017 u5        17               0.069538

In [54]:
res_[('country', 'year', 'age_group', 'cause')].tail()

death_rate  deaths  fraction_of_deaths
country year age_group cause                                        
zwe     2013 u5        17            NaN     NaN            0.067116
        2014 u5        17            NaN     NaN            0.068167
        2015 u5        17            NaN     NaN            0.068754
        2016 u5        17            NaN     NaN            0.072404
        2017 u5        17            NaN     NaN            0.069538

In [55]:
for k, v in res_.items():
    by = '--'.join(k)
    for c in v:
        # indicator = v.columns[0]
        v[[c]].dropna().to_csv(f'../../ddf--datapoints--{c}--by--{by}.csv')

In [56]:
# entities

In [57]:
e_gbl = pd.DataFrame({'global': ['global'], 'name': ['global'], 'is--global': ['TRUE']})

In [58]:
e_gbl

,global,name,is--global
0,global,global,TRUE


In [59]:
e_reg = reg[['region', 'whoreg6']].dropna().drop_duplicates()

In [60]:
e_reg

,region,whoreg6
18,5_emr,5_Emr
19,6_wpr,6_Wpr
20,1_afr,1_Afr
21,2_amr,2_Amr
22,3_sear,3_Sear
23,4_eur,4_Eur


In [61]:
e_reg.columns = ['region', 'name']

In [62]:
e_reg['is--region'] = 'TRUE'

In [63]:
e_country = country[['country', 'whoname', 'iso3', 'whocode', 'whoreg6']].dropna().drop_duplicates(subset='country')

In [64]:
e_country

,country,whoname,iso3,whocode,whoreg6
126,afg,Afghanistan,AFG,3010.0,5_Emr
144,ago,Angola,AGO,1020.0,1_Afr
162,alb,Albania,ALB,4005.0,4_Eur
180,and,Andorra,AND,4008.0,4_Eur
198,are,United Arab Emirates,ARE,3405.0,5_Emr
216,arg,Argentina,ARG,2020.0,2_Amr
234,arm,Armenia,ARM,4007.0,4_Eur
252,atg,Antigua and Barbuda,ATG,2010.0,2_Amr
270,aus,Australia,AUS,5020.0,6_Wpr
288,aut,Austria,AUT,4010.0,4_Eur


In [65]:
e_country['region'] = e_country['whoreg6'].str.lower()

In [66]:
e_country

,country,whoname,iso3,whocode,whoreg6,region
126,afg,Afghanistan,AFG,3010.0,5_Emr,5_emr
144,ago,Angola,AGO,1020.0,1_Afr,1_afr
162,alb,Albania,ALB,4005.0,4_Eur,4_eur
180,and,Andorra,AND,4008.0,4_Eur,4_eur
198,are,United Arab Emirates,ARE,3405.0,5_Emr,5_emr
216,arg,Argentina,ARG,2020.0,2_Amr,2_amr
234,arm,Armenia,ARM,4007.0,4_Eur,4_eur
252,atg,Antigua and Barbuda,ATG,2010.0,2_Amr,2_amr
270,aus,Australia,AUS,5020.0,6_Wpr,6_wpr
288,aut,Austria,AUT,4010.0,4_Eur,4_eur


In [67]:
e_country.columns

Index(['country', 'whoname', 'iso3', 'whocode', 'whoreg6', 'region'], dtype='object')

In [68]:
e_country = e_country[['country', 'whoname', 'iso3', 'whocode', 'region']].copy()

In [69]:
e_country['whocode'] = e_country['whocode'].map(int)

In [70]:
e_country.columns = ['country', 'name', 'iso3', 'whocode', 'region']

In [71]:
e_country['is--country'] = "TRUE"

In [72]:
e_gbl.to_csv('../../ddf--entities--geo--global.csv', index=False)

In [73]:
e_reg.to_csv('../../ddf--entities--geo--region.csv', index=False)

In [74]:
e_country.to_csv('../../ddf--entities--geo--country.csv', index=False)

In [75]:
e_age = pd.DataFrame([
    ['neo', 'neonatal period'],
    ['post', 'postneonatal period'],
    ['u5', 'under five (i.e., neo+post)']
], columns=['age_group', 'name'])

In [77]:
e_age.to_csv('../../ddf--entities--age_group.csv', index=False)

In [78]:
e_causes = dict(cats)

In [80]:
e_causes = pd.DataFrame.from_dict(e_causes, orient='index')

In [82]:
e_causes = e_causes.reset_index()

In [83]:
e_causes.columns = ['cause', 'name']

In [84]:
e_causes.to_csv('../../ddf--entities--cause.csv', index=False)

In [ ]:
# concepts

In [85]:
c_ent = pd.DataFrame([
    ['geo', 'Geo', 'entity_domain', ''],
    ['global', 'Global', 'entity_set', 'geo'],
    ['region', 'World in 6 regions', 'entity_set', 'geo'],
    ['country', 'Country', 'entity_set', 'geo'],
    ['cause', 'Causes', 'entity_domain', ''],
    ['age_group', 'Age Groups', 'entity_domain', '']
], columns=['concept', 'name', 'concept_type', 'domain'])

In [86]:
c_ent

,concept,name,concept_type,domain
0,geo,Geo,entity_domain,
1,global,Global,entity_set,geo
2,region,World in 6 regions,entity_set,geo
3,country,Country,entity_set,geo
4,cause,Causes,entity_domain,
5,age_group,Age Groups,entity_domain,


In [94]:
c_str = pd.DataFrame([
    ['name', 'Name', 'string'],
    ['domain', 'Domain', 'string'],
    ['whocode', 'WHO Code', 'string'],
    ['iso3', 'ISO 3', 'string'],
    ['year', 'Year', 'time']
], columns=['concept', 'name', 'concept_type'])

In [101]:
c_dps = pd.DataFrame([
    ['deaths', 'Total deaths'],
    ['death_rate', 'Death rate'],
    ['fraction_of_deaths', 'fraction of deaths'],
    ['live_births', 'Live Births']
], columns=['concept', 'name'])

c_dps['concept_type'] = 'measure'

In [102]:
cdf = pd.concat([c_dps, c_ent, c_str], sort=False)

In [103]:
cdf.to_csv('../../ddf--concepts.csv', index=False)